In [ ]:
import json
import numpy as np
from pycocotools import mask
from skimage import measure
import cv2

#Find convex hull and draw convex hull with maximum area. This helps to remove the small noisy areas in the image
import matplotlib.pyplot as plt
def find_hull(prediction):  # Input the segmented image from the model. Function outputs the convex hull with maximum area.
    prediction = prediction.astype(np.uint8) 
    contours, hierarchy = cv2.findContours(prediction, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # Find contours using opencv
    if len(contours)!=0:
        hull = []
        for i in range(len(contours)): # Find convex hull for each contour set
            hull.append(cv2.convexHull(contours[i], False))
        drawing = np.zeros((prediction.shape[0], prediction.shape[1], 3)) # Create an empty black image
        color = (255, 255, 255) # hull color
        c = max(hull, key = cv2.contourArea) # Find hull with maximum area
        m = []
        m.append(c)
        return m

for i in range(0,3):
    hull = []
    path = '/Users/sonudileep/Downloads/Mask_606/' + str(i) + '.png' 
    ground_truth_binary_mask = cv2.imread(path,0)
    ground_truth_binary_mask = ground_truth_binary_mask/255
    ground_truth_binary_mask = ground_truth_binary_mask.astype(np.uint8)
    countzero_in1 = not np.any(ground_truth_binary_mask)
    print(i, countzero_in1)
    if countzero_in1==True:
        continue    
    hull = find_hull(ground_truth_binary_mask)
    #print(hull)
    fortran_ground_truth_binary_mask = np.asfortranarray(ground_truth_binary_mask)
    encoded_ground_truth = mask.encode(fortran_ground_truth_binary_mask)
    ground_truth_area = mask.area(encoded_ground_truth)
    ground_truth_bounding_box = mask.toBbox(encoded_ground_truth)
    contours = measure.find_contours(ground_truth_binary_mask, 0.5)
    null = 'null'
    annotation = {
            "version": "4.5.6",
            "flags": {},
            "shapes": [{ "label" : "1",
                        "points" : " ",
                        "group_id": null,
                        "shape_type": "polygon",
                        "flags": {} 
                        }],
            "imagePath": ""       
        }
    yy = 0
#     for contour in contours:
#         contour = np.flip(contour, axis=1)
#         segmentation = contour.ravel().tolist()
#         print(type(segmentation))
    temp = []
    points = []
    for k in range(len(segmentation)):
        if k%2==0:
            temp = []
        temp.append(segmentation[k])
        if k%2==1:
            points.append(temp)
    print(type(points[0][0]))
#    print(points)
    annotation["shapes"][0]["points"] = points
    annotation["imagePath"]= str(i) + ".jpg"
#    print(annotation)
    path_1 = '/Users/sonudileep/Downloads/Mask_606/' + str(i) + '.json'
    m = json.dumps(annotation, indent=4)
    json_path = "/Users/sonudileep/Downloads/Mask_606/" + str(i) + ".json"
    text_file = open(json_path, "w")
    n = text_file.write(m)
    text_file.close()

0 True
1 True
2 False
<class 'float'>


In [ ]:

i = 2
path = '/Users/sonudileep/Downloads/Mask_606/' + str(i) + '.png' 
ground_truth_binary_mask = cv2.imread(path,0)
ground_truth_binary_mask = ground_truth_binary_mask/255
hull = find_hull(ground_truth_binary_mask)

[array([[[388,   0]],

       [[388,   1]],

       [[386,   6]],

       [[201, 409]],

       [[198, 415]],

       [[ 20, 415]],

       [[ 38, 364]],

       [[ 43, 350]],

       [[ 46, 343]],

       [[ 53, 327]],

       [[113, 190]],

       [[157,  90]],

       [[194,   6]],

       [[197,   0]]], dtype=int32)]


In [ ]:
!pip install pycocotools

In [ ]:
!pip install git+https://github.com/waleedka/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI

In [ ]:
import os
import io
import json
import re
from pycocotools.cocostuffhelper import pngToCocoResult

def pngToCocoResultDemo(dataDir='../..', resType='examples', indent=None):
    '''
    Converts a folder of .png images with segmentation results back
    to the COCO result format. 
    :param dataDir: location of the COCO root folder
    :param resType: identifier of the result annotation file
    :param indent: number of whitespaces used for JSON indentation
    :return: None
    '''

    # Define paths
    pngFolder = '%s/results/segmentations/%s' % (dataDir, resType)
    jsonPath = '%s/results/stuff_%s_results.json' % (dataDir, resType)

    # Get images in png folder
    imgNames = os.listdir(pngFolder)
    imgNames = [imgName[:-4] for imgName in imgNames if imgName.endswith('.png')]
    imgNames.sort()
    imgCount = len(imgNames)

    # Init
    annCount = 0

    with io.open(jsonPath, 'w', encoding='utf8') as output:
        print('Writing results to: %s' % jsonPath)

        # Annotation start
        output.write(unicode('[\n'))

        for i, imgName in zip(xrange(0, imgCount), imgNames):
            print('Converting png image %d of %d: %s' % (i+1, imgCount, imgName))

            # Add stuff annotations
            pngPath = '%s/%s.png' % (pngFolder, imgName)
            tokens = imgName.split('_')
            if len(tokens) == 1:
                # COCO 2017 format
                imgId = int(imgName)
            elif len(tokens) == 3:
                # Previous COCO format
                imgId = int(tokens[2])
            else:
                raise Exception('Error: Invalid COCO file format!')

            anns = pngToCocoResult(pngPath, imgId)

            # Write JSON
            str_ = json.dumps(anns, indent=indent)
            str_ = str_[1:-1]
            if len(str_) > 0:
                output.write(unicode(str_))
                annCount = annCount + 1

            # Add comma separator
            if i < imgCount-1 and len(str_) > 0:
                output.write(unicode(','))

            # Add line break
            output.write(unicode('\n'))

        # Annotation end
        output.write(unicode(']'))

        # Create an error if there are no annotations
        if annCount == 0:
            raise Exception('The output file has 0 annotations and will not work with the COCO API!')

if __name__ == "__main__":
    pngToCocoResultDemo('/Users/sonudileep/Downloads/Segmentation-Outputs ')

In [ ]:
import numpy as np
from imantics import Polygons, Mask

i = 2
path = '/Users/sonudileep/Downloads/Segmentation-Outputs/' + str(i) + '.png' 
ground_truth_binary_mask = cv2.imread(path,0)
ground_truth_binary_mask = ground_truth_binary_mask/255
array = ground_truth_binary_mask.astype(np.uint8)
# This can be any array
#array = np.ones((100, 100))

polygons = Mask(array).polygons()

print(polygons.points)
print(polygons.segmentation)

In [ ]:
len(polygons.points[0])

4878

In [ ]:
import json
import numpy as np
from pycocotools import mask
from skimage import measure
import cv2

#Find convex hull and draw convex hull with maximum area. This helps to remove the small noisy areas in the image
import matplotlib.pyplot as plt
def find_hull(prediction):  # Input the segmented image from the model. Function outputs the convex hull with maximum area.
    prediction = prediction.astype(np.uint8) 
    contours, hierarchy = cv2.findContours(prediction, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # Find contours using opencv
    if len(contours)!=0:
        hull = []
        for i in range(len(contours)): # Find convex hull for each contour set
            hull.append(cv2.convexHull(contours[i], False))
        drawing = np.zeros((prediction.shape[0], prediction.shape[1], 3)) # Create an empty black image
        color = (255, 255, 255) # hull color
        c = max(hull, key = cv2.contourArea) # Find hull with maximum area
        m = []
        m.append(c)
        return m

for i in range(0,147):
    hull = []
    path = '/Users/sonudileep/Downloads/Mask_606/' + str(i) + '.png' 
    ground_truth_binary_mask = cv2.imread(path,0)
    ground_truth_binary_mask = ground_truth_binary_mask/255
    ground_truth_binary_mask = ground_truth_binary_mask.astype(np.uint8)
    countzero_in1 = not np.any(ground_truth_binary_mask)
    #print(i, countzero_in1)
    if countzero_in1==True:
        continue    
    hull = find_hull(ground_truth_binary_mask)
    #print(hull)
    fortran_ground_truth_binary_mask = np.asfortranarray(ground_truth_binary_mask)
    encoded_ground_truth = mask.encode(fortran_ground_truth_binary_mask)
    ground_truth_area = mask.area(encoded_ground_truth)
    ground_truth_bounding_box = mask.toBbox(encoded_ground_truth)
    contours = measure.find_contours(ground_truth_binary_mask, 0.5)
    null = 'null'
    annotation = {
            "version": "4.5.6",
            "flags": {},
            "shapes": [{ "label" : "1",
                        "points" : " ",
                        "group_id": null,
                        "shape_type": "polygon",
                        "flags": {} 
                        }],
            "imagePath": ""       
        }
    yy = 0
#     for contour in contours:
#         contour = np.flip(contour, axis=1)
#         segmentation = contour.ravel().tolist()
#         print(type(segmentation))
    temp = []
    points = []
    #print(hull[0].shape[0])
    for k in range(hull[0].shape[0]):
#        print(hull[0][k][0][0], hull[0][k][0][1])
        points.append([float(hull[0][k][0][0]), float(hull[0][k][0][1])])
  #      points.append(hull[0][k][0])
   # print(points)
    annotation["shapes"][0]["points"] = points
    annotation["imagePath"]= str(i) + ".jpg"
#    print(annotation)
    path_1 = '/Users/sonudileep/Downloads/Mask_606/' + str(i) + '.json'
    m = json.dumps(annotation, indent=4)
    json_path = "/Users/sonudileep/Downloads/Image_606/" + str(i) + ".json"
    text_file = open(json_path, "w")
    n = text_file.write(m)
    text_file.close()

388 0
388 1
386 6
201 409
198 415
20 415
38 364
43 350
46 343
53 327
113 190
157 90
194 6
197 0
344 0
344 15
332 408
331 415
125 415
125 411
147 0
275 0
275 2
267 34
264 40
261 44
258 47
254 50
251 52
249 53
245 54
240 54
236 52
234 50
232 47
230 43
229 40
228 35
228 26
229 19
230 13
232 2
233 0
232 0
231 24
230 29
227 30
223 30
219 24
215 17
210 8
206 0
235 0
236 9
238 168
239 256
240 353
240 415
209 415
208 73
208 0
94 0
96 2
112 21
153 70
240 174
311 259
411 379
415 384
415 415
372 415
369 412
308 345
208 235
189 214
180 204
147 167
12 15
5 7
0 1
0 0
1 36
5 39
405 344
414 351
415 352
415 410
414 410
3 141
0 139
0 36
295 0
295 11
294 50
285 377
284 413
283 415
141 415
141 392
147 240
156 14
157 0
250 0
248 11
242 43
189 325
176 394
172 415
48 415
48 410
49 403
53 376
87 148
106 21
109 1
110 0
293 0
294 15
301 156
301 181
297 193
295 197
292 202
290 205
286 210
282 214
279 216
274 219
268 221
251 221
246 219
240 216
234 210
230 204
227 199
134 0
331 1
331 6
330 13
328 26
318 89
314 11